## Allowance(Coops and Deals) Data Cleasing

##### Data Sources: DB2 and SAP
##### Markets: 
-               US
-               CA
-               CN
-               IN (No deals in India market)
-               AR (Deal amount data for AR are not accessible)

### 01. Fill Deal Amount Data to Master Data (DB2)
1.1 In SAP Data, columns names are different

1.2 Import DB2 Data

1.3 Fill Deal amount data from SAP to the DB2 master table

1.4 Validate SAP amount records

### 02. Process Exchange Rate Data

### 03. Derived additional datafields (columns) based on raw data and reference data
3.1 Process the 32-38 datafields 

3.2 Process the 39-45 datafields 

3.3 Process the 46-53 datafields 

3.4 Process the 53-55 datafields 

3.5 Process the 56-57 datafields (Risk Indicators)

In [1]:
import pandas as pd
import numpy as np
from datetime import date
import os

### 01. Fill Deal Amount Data to Master Data (DB2)

###### 1.1 In SAP Data, columns names are different
-  1) Deal_ID is named as "Reference"
-  2) Amount is named 'Amount in doc. curr.'(CN),'Amount in local currency'(US,CA)
-  3) 1 Deal_ID have multiple rows, because of different posting dates


      





In [2]:
## Set up diretory, where the SAP data is stored

os.chdir(r'\\phont80025us.homeoffice.wal-mart.com\Shares\Audit_Analytics\FY19\Projects\Global Vendor Allowance\Data\Coop_Deal_Raw_Data\SAP_Deal_Amt')



# Import SAP Data
# because the format or the volume of the data, it takes time to read in below 3 files
df_01 = pd.ExcelFile('VA_CN_SAP.xlsx').parse('Sheet1')
df_02 = pd.ExcelFile('VA_CAN_SAP.xlsx').parse('Sheet1')
df_03 = pd.ExcelFile('VA_US_SAP_A110.xlsx').parse('Sheet1')



## opening the origin SAP file will make the excel disfunction
## maybe because the format or the volume of the data
## in below I write the SAP data to new formats to make the file openable, so that validation could be conducted easily

# for the last row of the origin data is total amount, remove that row
df_01[0:-1].to_excel('VA_CN_Deal.xlsx', sheet_name='sheet1', index=False)
df_02[0:-1].to_excel('VA_CAN_Deal.xlsx', sheet_name='sheet1', index=False)
df_03[0:-1].to_excel('VA_US_Deal.xlsx', sheet_name='sheet1', index=False)

In [3]:
###### Creat function to summarize total amount for each allowance ID
def deal_sum(df,amt_col_name,country_name):
    df_deal_amt = pd.pivot_table(df,
                                 index = ['Reference'],
                                 values = [amt_col_name], # e.g. 'Amount in doc. curr.'
                                 aggfunc={amt_col_name:np.sum})
    # convert the indices to column
    df_deal_amt = df_deal_amt.reset_index() 
    
    # rename the columns of the data frame
    df_deal_amt.columns = ['DEAL_ID','AMOUNT_SAP']
    
    # creat a new column to identify the contry name, for 3 SAP summarized data needed to be concate later
    df_deal_amt['COUNTRY'] = country_name
    
    # Change the data type of DEAL_ID
    df_deal_amt['DEAL_ID'] = df_deal_amt['DEAL_ID'].apply(lambda x: int(x))
    return (df_deal_amt)

# summarize total amount for each allowance ID for each country
df_deal_CN  = deal_sum (df_01[0:-1],'Amount in doc. curr.','CN')
df_deal_CA  = deal_sum (df_02[0:-1],'Amount in local currency','CA')
df_deal_US  = deal_sum (df_03[0:-1],'Amount in local currency','US')

# concate deal amount data to 1 file
df_deal_all = pd.concat([df_deal_CN,df_deal_CA,df_deal_US],ignore_index = True)

###### 1.2 Import DB2 Data

In [9]:
## Set up diretory, where the DB2 Data is stored

os.chdir(r'\\phont80025us.homeoffice.wal-mart.com\Shares\Audit_Analytics\FY19\Projects\Global Vendor Allowance\Data\Coop_Deal_Raw_Data\DB2')

# import 5 files in such folder all at once
# because the volume of the data, it takes time to read in all files in the folder

files = os.listdir(r'\\phont80025us.homeoffice.wal-mart.com\Shares\Audit_Analytics\FY19\Projects\Global Vendor Allowance\Data\Coop_Deal_Raw_Data\DB2')

frames = {f[:2]:pd.ExcelFile(f).parse('Sheet1')
          for f in files if f.endswith('.xlsx')}


In [10]:
# Concate 5 files to 1 file, so that no need to join and merge multiple times
db2_CoopDeal = pd.concat([frames[i] for i in list(frames.keys())],ignore_index = True)

###### 1.3 Fill Deal amount data from SAP to the DB2 master table
- 1 deal may have mulitple row of records  because the amount could be allocated to mulitple departments
- In other words, DEAL_IDs are the same, but 'VENDR_DEPT_NBR's aren't

- When filling deal amount to DB2 master table, if the deal id have mulitiple rows,  the amount will be the same

In [11]:
df_db2_SAP = pd.merge(db2_CoopDeal,df_deal_all,how = 'left')

In [15]:
ix =  df_db2_SAP[(df_db2_SAP['DEAL_OR_COOP'] =='D')&(df_db2_SAP['AMOUNT_SAP'].notnull())].index

df_db2_SAP.loc[ix,'AMOUNT'] = df_db2_SAP.loc[ix,'AMOUNT_SAP'] 

In [ ]:
### Write the consolidated allowance data 
### So that no need to rerun the code from the every beginning
os.chdir(r'\\phont80025us.homeoffice.wal-mart.com\Shares\Audit_Analytics\FY19\Projects\Global Vendor Allowance\Data\Coop_Deal_Raw_Data\DB2')

df_db2_SAP.to_csv('Coop_Deal_DB2_SAP.csv', index = False,encoding='utf_8_sig')

######  1.4 Validate SAP amount records
- if any deal id in DB2 is not in SAP
- if any deal id in SAP is not in DB2

In [16]:
# Check unique Deal IDs in SAP and DB2 Data, to see if any deal id in DB2 is not in SAP

db2_Deal_IDs = np.unique(db2_CoopDeal[db2_CoopDeal['DEAL_OR_COOP'] =='D']['DEAL_ID'])
SAP_Deal_IDs = np.unique(df_deal_all['DEAL_ID'])

print('Unique Deal ID in DB2',len(db2_Deal_IDs ))
print('Unique Deal ID in SAP',len(SAP_Deal_IDs))


# Deals In DB2 not in SAP
Deal_No_SAP = list(set(db2_Deal_IDs) - set(SAP_Deal_IDs))
Deal_No_DB2 = list(set(SAP_Deal_IDs) - set(db2_Deal_IDs))

print(' '.join([str(len(Deal_No_DB2)),'Deals exist in SAP but NOT in DB2']))
print(' '.join([str(len(Deal_No_SAP)),'Deals exist in DB2 but NOT in SAP']))



('Unique Deal ID in DB2', 84968)
('Unique Deal ID in SAP', 72922)
19609 Deals exist in SAP but NOT in DB2
31655 Deals exist in DB2 but NOT in SAP


In [17]:
df_No_Deal_amt =df_db2_SAP[(df_db2_SAP['DEAL_OR_COOP'] =='D')&(df_db2_SAP['AMOUNT_SAP'].isnull())]

No_Deal_amt_Country = pd.pivot_table(df_No_Deal_amt,
                                 index = ['COUNTRY'],
                                 values = ['DEAL_ID'], # e.g. 'Amount in doc. curr.'
                                 aggfunc={'DEAL_ID':'count'})
No_Deal_amt_Country = No_Deal_amt_Country.reset_index()


df_Deal_all = df_db2_SAP[(df_db2_SAP['DEAL_OR_COOP'] =='D')]
Deal_Country = pd.pivot_table(df_Deal_all,
                                 index = ['COUNTRY'],
                                 values = ['DEAL_ID'], # e.g. 'Amount in doc. curr.'
                                 aggfunc={'DEAL_ID':'count'})
Deal_Country = Deal_Country.reset_index()

Deal_NoAmt_Sum = pd.merge(Deal_Country,No_Deal_amt_Country,how = 'left', on ='COUNTRY')

Deal_NoAmt_Sum.rename(columns={'DEAL_ID_x': 'Tot_Deal_Count',
                               'DEAL_ID_y': 'Deal_NoAmt_Count'}, inplace=True)

Deal_NoAmt_Sum['NoAmt_%'] = Deal_NoAmt_Sum['Deal_NoAmt_Count']/Deal_NoAmt_Sum['Tot_Deal_Count']*100

Deal_NoAmt_Sum

,COUNTRY,Tot_Deal_Count,Deal_NoAmt_Count,NoAmt_%
0,AR,10498,10498,100.000000
1,CA,35775,3289,9.193571
2,CN,36059,16719,46.365678
3,US,3075,1447,47.056911


### 02. Process Exchange Rate Data
- Average monthly exchange rates are used to convert local currency to US dollar
- Average monthly exchange rates are provided by Finance team
- Link for files of average monthly exchange rates: https://walmartglobal.box.com/s/ahnbopxzvgy3kr1fv5uvqs6mem5l779t
- files in such link were downloaded to this folder: 

'\\phont80025us.homeoffice.wal-mart.com\Shares\Audit_Analytics\FY19\Projects\Global Vendor Allowance\Data\Foriegn Exchange rates


In [18]:
## Set up diretory, where the files of average monthly exchange rates are stored

os.chdir(r'\\phont80025us.homeoffice.wal-mart.com\Shares\Audit_Analytics\FY19\Projects\Global Vendor Allowance\Data\Foriegn Exchange rates')


# import 25 files in such folder all at once
# because the volume of the data, it takes time to read in all files in the folder

files = os.listdir(r'\\phont80025us.homeoffice.wal-mart.com\Shares\Audit_Analytics\FY19\Projects\Global Vendor Allowance\Data\Foriegn Exchange rates')

df_exrate = [f for f in files if f.endswith('.xlsx')]

In [19]:
# Concate all the 25 files

curr_list =['ARS','CAD','CLP','CNY','CRC','GBP' ,'GTQ' ,'HNL' ,'INR' ,'JPY','MXN','NIO','SVC','USD']
country_list = ['AR','CA','CL','CN','CR','GB','GT','HN','IN','JP','MX','NI','SV']

df_ex_GVA = pd.DataFrame()
for i in range(len(df_exrate)):
    df_ex = pd.ExcelFile(df_exrate[i]).parse('Y - avg rate')
    ix =df_ex.iloc[4:,0][df_ex.iloc[4:,0].apply(lambda x:x.rstrip()).apply(lambda x: x in curr_list )].index
    df_ex_new = df_ex.iloc[ix,[0,1]]
    df_ex_new['COUNTRY'] = country_list
    df_ex_new['Rate_Time'] = df_ex.iloc[3,1]
    df_ex_new.columns = ['Currency','Ex_Rate','COUNTRY','Time']
    
    df_ex_GVA = pd.concat([df_ex_GVA,df_ex_new],ignore_index = True)


In [ ]:
# Write the consolidate Exchange Rate file to this folder: "Reference Tables", for later reference
# \\phont80025us.homeoffice.wal-mart.com\Shares\Audit_Analytics\FY19\Projects\Global Vendor Allowance\Data\Reference Tables

os.chdir(r'\\phont80025us.homeoffice.wal-mart.com\Shares\Audit_Analytics\FY19\Projects\Global Vendor Allowance\Data\Reference Tables')
df_ex_GVA.to_csv('Exchange_Rate_GVA_2018.csv', index = False,encoding='utf_8_sig')

### 03. Derived additional datafields (columns) based on raw data and reference data
- for derived additional datafields explanation, refer to this file "Data Resources.xlsx"


- in this folder :\\phont80025us.homeoffice.wal-mart.com\Shares\Audit_Analytics\FY19\Projects\Global Vendor Allowance\Query_DB2_PO


- datafields are derived based on the sequence in Data Resources.xlsx

######  3.1 Process the 32-38 datafields 

In [20]:
## 31 - Year
## 32 - Month

df_db2_SAP['Year'] = df_db2_SAP['DEAL_STATUS_DATE'].apply(lambda x: x.year)
df_db2_SAP['Month'] = df_db2_SAP['DEAL_STATUS_DATE'].apply(lambda x: x.month)
df_ex_GVA['Month'] = df_ex_GVA['Time'].apply(lambda x: x.month)
df_ex_GVA['Year'] = df_ex_GVA['Time'].apply(lambda x: x.year)

In [25]:
## 33 - 'First_Yr'

# Yr_1: 08/01/2016 – 07/31/2017
# Yr_2:08/01/2017 – 07/31/2018
    
df_db2_SAP['First_Yr'] = 'Yr_2'

df_db2_SAP.loc[df_db2_SAP[df_db2_SAP['Year']<=2016].index,'First_Yr'] ='Yr_1'
    
ix_1 = df_db2_SAP[(df_db2_SAP['Year'] ==2017) & (df_db2_SAP['Month'] <=7)].index
df_db2_SAP.loc[ix_1,'First_Yr'] ='Yr_1'

In [26]:
## 35 Ex_Rate
## 36 Time
## 37 Currency

# set a different name for the new dataframe in case the merge result is incorrect
df_db2_SAP_1 = pd.merge(df_db2_SAP,df_ex_GVA,
                        how = 'left',on=['COUNTRY','Year','Month'])

In [27]:
# 38 AMT_US_Dollar
df_db2_SAP_1['AMT_US_Dollar'] =df_db2_SAP_1['AMOUNT']/df_db2_SAP_1['Ex_Rate']

####### Attention: for some reason, the 'Ex_Rate' is not available 
####### 1) if the allowance is for US market
####### 2) the months deal_status_date is not included in the Exchange Rate Data 

######  3.2 Process the 39-45 datafields 

- in this section, multiple referece data/look up tables needed to be imported


1) VA_Group_GVA_2018_11_6 DC.xlsx


2) Status_Group_GVA.xlsx


3) Dept_Desc_WM1_WW_CORE_DIM_VM.xlsx (This table contains department descriptions) 

######  3.2.1 39 - 40 Allowance Group & Volume/Rebates

In [28]:
# 39 Allowance Group
# 40 Volume/Rebates

# Import the reference table created by GVA team : VA_Group_GVA_2018_11_6 DC

os.chdir(r'\\phont80025us.homeoffice.wal-mart.com\Shares\Audit_Analytics\FY19\Projects\Global Vendor Allowance\Data\Reference Tables')
df_ref_1 =  pd.ExcelFile('VA_Group_GVA_2018_11_6 DC.xlsx').parse('Sheet3')

# set a different name for the new dataframe in case the merge result is incorrect
df_db2_SAP_2 = pd.merge(df_db2_SAP_1,
                        df_ref_1[['ALLOWANCE_TYPE_Ori','Allowance Group','Volume/Rebates']],
                        how = 'left',
                        left_on='ALLOWANCE_TYPE', right_on='ALLOWANCE_TYPE_Ori')

In [29]:
# Validate the datapoints in 'Allowance Group'

print("Below ALLOWANCE_TYPE are not categorized")
df_db2_SAP_2[df_db2_SAP_2['Allowance Group'].isnull()][['COUNTRY','ALLOWANCE_TYPE']].drop_duplicates()

Below ALLOWANCE_TYPE are not categorized


,COUNTRY,ALLOWANCE_TYPE
28992,CA,None
77939,CN,Inbound transportation services fee
78478,CN,Vendor Support
78534,CN,Dishonesty Loss Compensation
79467,CN,Warehouse Packsell
79505,CN,Government Inspection relating Fee
81378,CN,Return Services Fee
82436,CN,Violate food safety operation rules
83554,CN,Cost Reduction
392688,CN,Membership Benefit Brochure


In [30]:
# Validate the datapoints in 'Volume/Rebates'
df_db2_SAP_2[df_db2_SAP_2['Volume/Rebates'].isnull()][['COUNTRY','ALLOWANCE_TYPE','Volume/Rebates']].drop_duplicates()

,COUNTRY,ALLOWANCE_TYPE,Volume/Rebates
28992,CA,None,NaN
77939,CN,Inbound transportation services fee,NaN
78478,CN,Vendor Support,NaN
78534,CN,Dishonesty Loss Compensation,NaN
79467,CN,Warehouse Packsell,NaN
79505,CN,Government Inspection relating Fee,NaN
81378,CN,Return Services Fee,NaN
82436,CN,Violate food safety operation rules,NaN
83554,CN,Cost Reduction,NaN
392688,CN,Membership Benefit Brochure,NaN


######  3.2.2  41 DEAL_STATUS

In [31]:
df_ref_2 =  pd.ExcelFile('Status_Group_GVA.xlsx').parse('sheet1')

In [32]:
ix1  = df_db2_SAP_2[df_db2_SAP_2['DEAL_OR_COOP'] =='C'].index

print("for COOP, there are multple data types for 'DEAL_STATUS_CODE' ")
print(df_db2_SAP_2.loc[ix1,'DEAL_STATUS_CODE'].apply(lambda x: type(x)).unique())

for COOP, there are multple data types for 'DEAL_STATUS_CODE' 
[<type 'unicode'> <type 'long'>]


In [33]:
ix2  = df_db2_SAP_2[df_db2_SAP_2['DEAL_OR_COOP'] =='D'].index

print(df_db2_SAP_2.loc[ix2,'DEAL_STATUS_CODE'].apply(lambda x: type(x)).unique())

[<type 'unicode'>]


In [34]:
# Since the 'DEAL_STATUS_CODE' extacted from DB2 are of different data types
# convert 'DEAL_STATUS_CODE' to the same data type --  String



ix  = df_db2_SAP_2[df_db2_SAP_2['DEAL_OR_COOP'] =='C'].index
df_db2_SAP_2.loc[ix,'DEAL_STATUS_CODE'] = df_db2_SAP_2.loc[ix,'DEAL_STATUS_CODE'].apply(lambda x: str(int(x)).strip())

ix = set(df_db2_SAP_2.index) - set(ix)
df_db2_SAP_2.loc[ix,'DEAL_STATUS_CODE'] = df_db2_SAP_2.loc[ix,'DEAL_STATUS_CODE'].apply(lambda x: x.strip())
df_ref_2['DEAL_STATUS_CODE']= df_ref_2['DEAL_STATUS_CODE'].apply(lambda x: str(x))


df_db2_SAP_3 = pd.merge(df_db2_SAP_2,df_ref_2,
                    how = 'left',
                    left_on = [u'DEAL_OR_COOP',u'DEAL_STATUS_CODE'],
                     right_on = [u'DEAL_OR_COOP',u'DEAL_STATUS_CODE'])


In [35]:
# Validate the deal status group 
df_db2_SAP_3[df_db2_SAP_3['DEAL_STATUS_y'].isnull()]

,COUNTRY,DEAL_ID,AMOUNT,DEAL_OR_COOP,DEAL_STATUS_DATE,ALLOWANCE_TYPE,BASE_DIV_NBR,ACCT_DIV_NBR,SAMS_CATG_NBR,BUYER_NAME,...,First_Yr,Currency,Ex_Rate,Time,AMT_US_Dollar,ALLOWANCE_TYPE_Ori,Allowance Group,Volume/Rebates,DEAL_STATUS_DESC_y,DEAL_STATUS_y


In [36]:
# clean up the column names after merging
df_db2_SAP_3 = df_db2_SAP_3.drop([u'ALLOWANCE_ID', 'ALLOWANCE_TYPE_Ori','DEAL_STATUS_DESC_y','DEAL_STATUS_x'],axis=1)

df_db2_SAP_3.rename(columns={u'DEAL_STATUS_DESC_x':'DEAL_STATUS_DESC',
                             'DEAL_STATUS_y': 'DEAL_STATUS'}, inplace=True)


######  3.2.3 42-45  Business_Format/DEPT_DESC/MDSE_SUBGROUP_DESC/MDSE_SEGMENT_DESC

In [37]:
#### 42 Business Format

df_db2_SAP_3.loc[:,'Business_Format'] ='WMT'

#SAMS
df_db2_SAP_3.loc[df_db2_SAP_3[df_db2_SAP_3['BASE_DIV_NBR']==18].index,'Business_Format'] = 'Sams'

#WMT.COM
bool1 = df_db2_SAP_3['ACCT_DIV_NBR']==46
df_db2_SAP_3.loc[df_db2_SAP_3[bool1].index,'Business_Format'] ='WMT.COM'

### Notice: 
# In india market, only sams exist in this time period 2016-08-01 to 2018-07-31


In [38]:
#### 43 - 45

# Import department description file

df_ref_3 = pd.ExcelFile('Dept_Desc_WM1_WW_CORE_DIM_VM.xlsx').parse('Sheet1')

# Column name for department number                        -- DEPT_NBR
# Column name for department description                   -- DEPT_DESC
# Column name for business units(walmart or sam's)         -- BASE_DIV_NBR
# Column name for country names                            -- COUNTRY_CODE
# Column name for MDSE subgroup                            -- MDSE_SUBGROUP_DESC
# Column name for MDSE segment                             -- MDSE_SEGMENT_DESC

#### MDSE segment > MDSE subgroup


In [39]:
df_dept = df_ref_3[df_ref_3['CURRENT_IND'] =='Y'][['COUNTRY_CODE','BASE_DIV_NBR','DEPT_NBR','DEPT_DESC','MDSE_SUBGROUP_DESC','MDSE_SEGMENT_DESC']]

In [40]:
df_db2_SAP_4 = pd.merge(df_db2_SAP_3,df_dept,how = 'left',
                        left_on = ['COUNTRY','BASE_DIV_NBR', 'VENDR_DEPT_NBR'],
                        right_on = ['COUNTRY_CODE','BASE_DIV_NBR', 'DEPT_NBR'])

# clean up the column names after merging
df_db2_SAP_4 = df_db2_SAP_4.drop(['COUNTRY_CODE','DEPT_NBR', ],axis=1)

######  3.3 Process the 46-53 datafields 

In [41]:
## Notice, in DB2 data format, 
##### some records of EVENT_END_DATE and EVENT_BEGIN_DATE is '?'

##### to clean data from other systems or from markets, be careful about the data type of the date

col = ['EVENT_BEGIN_DATE','EVENT_END_DATE','AR_APPROVAL_DATE',
       #'BUYER_APPROVAL_DATE', --  No Null value under this date
       'DMM_APPROVAL_DATE']

# to fill the null value or "?" in any date columns with "Unknown"

##### EVENT_BEGIN_DATE:      datetime.datetime
##### EVENT_END_DATE:        datetime.datetime
##### AR_APPROVAL_DATE:      datetime.datetime
##### BUYER_APPROVAL_DATE:   'pandas.tslib.Timestamp'
##### DMM_APPROVAL_DATE:     datetime.datetime

for i in col:
    
    df_db2_SAP_4.loc[:,i] = df_db2_SAP_4[i].fillna('Unknown') 
    
    ix = df_db2_SAP_4[df_db2_SAP_4[i]=='?'].index
    df_db2_SAP_4.loc[ix,i] ='Unknown'

#######################################################
## Function to calculate month gap 
#######################################################
def month_gap (date1,date2):
    if (date1=='Unknown')or (date2=='Unknown'):
        month_gap = 'Unknown'
        
    elif str(type(date2)) == "<class 'pandas.tslib.Timestamp'>":
        year_2 = date2.date().year
        month_2 = date2.date().month
        
        year_1 = date1.year
        month_1 = date1.month
        month_gap = (year_2-year_1)*12 + (month_2-month_1)  
        
    else:
        year_1 = date1.year
        month_1 = date1.month
        
        year_2 = date2.year
        month_2 = date2.month     
        month_gap = (year_2-year_1)*12 + (month_2-month_1)       
    return (month_gap)

#######################################################
## Function to calculate day gap 
#######################################################
def day_gap (date1,date2):
    if (date1=='Unknown')or (date2=='Unknown'):
        day_gap = 'Unknown'
    else: 
        day_gap = int((date2 - date1).days)
    return (day_gap)
        
#######################################################
## Function to apply functions 
#######################################################
def lag_cal(df,func,date1,date2):
    lag = [func(df[date1][i],df[date2][i]) for i in df.index]
    return(lag)


In [42]:
# 46 C_D_Duration : EVENT_END_DATE  - EVENT_BEGIN_DATE
df_db2_SAP_4['C_D_Duration'] = lag_cal(df_db2_SAP_4,month_gap,'EVENT_BEGIN_DATE','EVENT_END_DATE')

In [43]:
# 47 Mul_Month_Coop
df_db2_SAP_4['Mul_Month_Coop'] ='N'
bool1 = (df_db2_SAP_4['C_D_Duration']>=2) & (df_db2_SAP_4['DEAL_OR_COOP'] == 'C')
ix = df_db2_SAP_4[bool1].index
df_db2_SAP_4.loc[ix,'Mul_Month_Coop'] ='Y'

In [44]:
# 48 EBD_AR_Day
# 49 EED_AR_Day
df_db2_SAP_4['EBD_AR_Day'] = lag_cal(df_db2_SAP_4,day_gap,'EVENT_BEGIN_DATE','AR_APPROVAL_DATE')
df_db2_SAP_4['EED_AR_Day'] = lag_cal(df_db2_SAP_4,day_gap,'EVENT_END_DATE','AR_APPROVAL_DATE') 

In [45]:
# 50 EventDay_AR_Month
# 51 EventDay_Buyer_Month
# 52 EventDay_DMM_Month

ix_c = df_db2_SAP_4[df_db2_SAP_4['DEAL_OR_COOP'] == 'C'].index
ix_d = df_db2_SAP_4[df_db2_SAP_4['DEAL_OR_COOP'] == 'D'].index

df_db2_SAP_4['EventDay_AR_Month'] = np.nan
df_db2_SAP_4['EventDay_Buyer_Month'] = np.nan
df_db2_SAP_4['EventDay_DMM_Month'] = np.nan

df_db2_SAP_4.loc[ix_c,'EventDay_AR_Month'] = lag_cal(df_db2_SAP_4.loc[ix_c,:],month_gap,'EVENT_END_DATE','AR_APPROVAL_DATE')
df_db2_SAP_4.loc[ix_d,'EventDay_AR_Month'] = lag_cal(df_db2_SAP_4.loc[ix_d,:],month_gap,'EVENT_BEGIN_DATE','AR_APPROVAL_DATE')


df_db2_SAP_4.loc[ix_c,'EventDay_Buyer_Month'] = lag_cal(df_db2_SAP_4.loc[ix_c,:],month_gap,'EVENT_END_DATE','BUYER_APPROVAL_DATE')
df_db2_SAP_4.loc[ix_d,'EventDay_Buyer_Month'] = lag_cal(df_db2_SAP_4.loc[ix_d,:],month_gap,'EVENT_BEGIN_DATE','BUYER_APPROVAL_DATE')

df_db2_SAP_4.loc[ix_c,'EventDay_DMM_Month'] = lag_cal(df_db2_SAP_4.loc[ix_c,:],month_gap,'EVENT_END_DATE','DMM_APPROVAL_DATE')
df_db2_SAP_4.loc[ix_d,'EventDay_DMM_Month'] = lag_cal(df_db2_SAP_4.loc[ix_d,:],month_gap,'EVENT_BEGIN_DATE','DMM_APPROVAL_DATE')


######  3.4 Process the 53-55 datafields 


In [46]:
# 53 Old_Policy_Time
# Old Policy Execution Period: before Oct 31, 2017
df_db2_SAP_4['Old_Policy_Time'] = 'N'
df_db2_SAP_4.loc[df_db2_SAP_4[df_db2_SAP_4['Year']<=2016].index,'Old_Policy_Time'] ='Y'    
ix_1 = df_db2_SAP_4[(df_db2_SAP_4['Year'] ==2017) & (df_db2_SAP_4['Month'] <=10)].index
df_db2_SAP_4.loc[ix_1,'Old_Policy_Time'] ='Y'

In [47]:
# 54 Compliant_or_Not

df_db2_SAP_4['Compliant_or_Not'] =np.nan
# have DMM signiture
bool_10 = (df_db2_SAP_4['EventDay_DMM_Month'] != 'Unknown')
bool_11 = df_db2_SAP_4['EventDay_DMM_Month']<=0
bool_12 = (df_db2_SAP_4['EventDay_DMM_Month']>0)&(df_db2_SAP_4['EventDay_DMM_Month']<=1)
bool_13 = df_db2_SAP_4['EventDay_DMM_Month']>=2

df_db2_SAP_4.loc[df_db2_SAP_4[bool_10 & bool_11].index,'Compliant_or_Not'] = 'Compliant'
df_db2_SAP_4.loc[df_db2_SAP_4[bool_10 & bool_12].index,'Compliant_or_Not'] = 'TBD'
df_db2_SAP_4.loc[df_db2_SAP_4[bool_10 & bool_13].index,'Compliant_or_Not'] = 'Not Compliant'

# have AR signiture, but no DMM signiture
bool_10 = (df_db2_SAP_4['EventDay_DMM_Month'] == 'Unknown') & (df_db2_SAP_4['EventDay_AR_Month'] != 'Unknown')
bool_11 = df_db2_SAP_4['EventDay_AR_Month']<=0
bool_12 = (df_db2_SAP_4['EventDay_AR_Month']>0)&(df_db2_SAP_4['EventDay_AR_Month']<=1)
bool_13 = df_db2_SAP_4['EventDay_AR_Month']>=2

df_db2_SAP_4.loc[df_db2_SAP_4[bool_10 & bool_11].index,'Compliant_or_Not'] = 'Compliant'
df_db2_SAP_4.loc[df_db2_SAP_4[bool_10 & bool_12].index,'Compliant_or_Not'] = 'TBD'
df_db2_SAP_4.loc[df_db2_SAP_4[bool_10 & bool_13].index,'Compliant_or_Not'] = 'Not Compliant'

# only have buyer signiture
bool_10 = (df_db2_SAP_4['EventDay_DMM_Month'] == 'Unknown')&(df_db2_SAP_4['EventDay_AR_Month'] == 'Unknown')
bool_11 =df_db2_SAP_4['EventDay_Buyer_Month']<=0
bool_12 = (df_db2_SAP_4['EventDay_Buyer_Month']>0)&(df_db2_SAP_4['EventDay_Buyer_Month']<=1)
bool_13 = df_db2_SAP_4['EventDay_Buyer_Month']>1

df_db2_SAP_4.loc[df_db2_SAP_4[bool_10 & bool_11].index,'Compliant_or_Not'] = 'Compliant'
df_db2_SAP_4.loc[df_db2_SAP_4[bool_10 & bool_12].index,'Compliant_or_Not'] = 'TBD'
df_db2_SAP_4.loc[df_db2_SAP_4[bool_10 & bool_13].index,'Compliant_or_Not'] = 'Not Compliant'


In [ ]:

df_db2_SAP_4['Compl_Old_Policy'] =np.nan

bool_D_old = (df_db2_SAP_4['DEAL_OR_COOP'] == 'D') & (df_db2_SAP_4['Old_Policy_Time'] =='Y') &(df_db2_SAP_4['EBD_AR_Day'] >= 16)
bool_C_old = (df_db2_SAP_4['DEAL_OR_COOP'] == 'C') & (df_db2_SAP_4['Old_Policy_Time'] =='Y') &(df_db2_SAP_4['EED_AR_Day']>= 31)


ix_D_old =  df_db2_SAP_4[bool_D_old].index
ix_C_old =  df_db2_SAP_4[bool_C_old].index


df_db2_SAP_4.loc[ix_D_old,'Compl_Old_Policy'] = 'Not Compliant'
df_db2_SAP_4.loc[ix_C_old,'Compl_Old_Policy'] = 'Not Compliant'

ix_Comp_old = df_db2_SAP_4[(df_db2_SAP_4['Old_Policy_Time'] =='Y') &(df_db2_SAP_4['Compl_Old_Policy'].isnull())].index
df_db2_SAP_4.loc[ix_Comp_old,'Compl_Old_Policy'] = 'Compliant'

ix_Comp_new = df_db2_SAP_4[df_db2_SAP_4['Compl_Old_Policy'].isnull()].index
df_db2_SAP_4.loc[ix_Comp_new,'Compl_Old_Policy'] = df_db2_SAP_4.loc[ix_Comp_new,'Compliant_or_Not']


###### 3.5 Process the 56-57 datafields 

In [ ]:
# 56 Fst2_digit

def fst2_digit_extraction(x1):
    x1 = str(x1).replace('.','') 
    digit = x1[0:2]
    return(digit)

df_db2_SAP_4['Fst2_digit']= df_db2_SAP_4['AMOUNT'].apply(lambda x:fst2_digit_extraction(abs(x)))


# fix amount less than 1, e.g 0.016
ix = df_db2_SAP_4[df_db2_SAP_4['Fst2_digit'].apply(lambda x: x.startswith('0'))].index
df_db2_SAP_4.loc[ix,'Fst2_digit'] = df_db2_SAP_4.loc[ix,'AMOUNT'].apply(lambda x: str(float(x)*100)[0:2])

# fix amount less than 1 and with less than 3 decimals, e.g 0.01
ix = df_db2_SAP_4[df_db2_SAP_4['Fst2_digit'].apply(lambda x: x.endswith('.'))].index
df_db2_SAP_4.loc[ix,'Fst2_digit'] = df_db2_SAP_4.loc[ix,'Fst2_digit'].apply(lambda x: int(x.split('.')[0])*10)

In [ ]:
# BFL_Prob
bfl_Fst2_digit= []
for i in range(1,10):
    for j in range(10):
        bfl_Fst2_digit = bfl_Fst2_digit + [int(str(i)+str(j))]
bfl_Fst2_digit_dist = [4.14,3.78,3.48,3.22,3.00,2.80,2.63,2.48,2.35,2.23,
                      2.12,2.02,1.93,1.85,1.77,1.70,1.64,1.58,1.52,1.47,
                      1.42,1.38,1.34,1.30,1.26,1.22,1.19,1.16,1.13,1.10,
                      1.07,1.05,1.02,1.00,0.98,0.95,0.93,0.91,0.90,0.88,
                       0.86,0.84,0.83,0.81,0.80,0.78,0.77,0.76,0.74,0.73,
                       0.72,0.71,0.69,0.68,0.67,0.66,0.65,0.64,0.63,0.62,
                       0.62,0.61,0.60,0.59,0.58,0.58,0.57,0.56,0.55,0.55,
                       0.54,0.53,0.53,0.52,0.51,0.51,0.50,0.50,0.49,0.49,
                       0.48,0.47,0.47,0.46,0.46,0.45,0.45,0.45,0.44,0.44]

digit_prob = pd.DataFrame()
digit_prob['Fst2_digit'] = bfl_Fst2_digit 
digit_prob['BFL_Prob']  =bfl_Fst2_digit_dist

df_db2_SAP_4 = pd.merge(df_db2_SAP_4,digit_prob,how = 'left',on ='Fst2_digit' )


###### 3.5.1 Process the 12 Risk Indicators

- R01 Tot_Appr_Levels
- R02 Dup_Authorization
- R03 Duplicate
- R04 Round_Dollar
- R05 Buyer_Lst_5_days
- R06 Vendor_Lst_5_days
- R07 DMM_Lst_5_days
- R08 AR_Lst_5_days
- R09 All_Lst_5_days
- R10 BuyerInAR
- R11 VendorInAR
- R12 DMMInAR

In [ ]:
# R01 Tot_Appr_Levels

ID_col = [i for i in df_db2_SAP_4.columns  if i.endswith('ID')]

for i in [2,3,4]:   
    
    # every allowances in DB2 has Buyer ID
    # but not necessarily have Vendor ID, DMM ID, AR ID, if the allowace has no such IDs, its value is '?'
    # when apply this for loop to data from other system, please check the IDs
   
    ix  = df_db2_SAP_4[df_db2_SAP_4[ID_col[i]] == '?'].index
    df_db2_SAP_4.loc[ix,ID_col[i]] = np.nan
    
    # use 'apply'function to a dataframe, use axis = 1, apply function rowwise
df_db2_SAP_4['R01_Tot_Appr_Levels'] = 4- df_db2_SAP_4[ID_col].isnull().apply(np.sum, axis=1)


In [ ]:
# R02 Dup_Authorization

df = pd.DataFrame()
for i in [1,2,3,4]:
    for j in [2,3,4]:
        if (i!=j)&(j>i): 
            col = '_vs_'.join([ID_col[i].split('_')[0],ID_col[j].split('_')[0]])
            df[col] = df_db2_SAP_4[ID_col[i]] == df_db2_SAP_4[ID_col[j]]
            
df_db2_SAP_4['R02 Dup_Authorization'] = df.apply(np.sum, axis=1)

In [ ]:
# R03 Duplicate

#same vendor,*same year, same month, same amount, *same deal status

Duplicate_1 = pd.pivot_table(df_db2_SAP_4,index = ['COUNTRY','VENDR_NBR','Month','Year','AMOUNT','DEAL_STATUS'],
                             values = ['DEAL_ID'],
                             aggfunc={u'DEAL_ID':'count'})
Duplicate_1 = Duplicate_1.reset_index()
Duplicate_1.rename(columns={'DEAL_ID': 'R03 Duplicate'},inplace=True) 
Duplicate_1 = Duplicate_1[(Duplicate_1['AMOUNT']!=0)&(Duplicate_1['R03 Duplicate']!=1)]
df_db2_SAP_4 = pd.merge(df_db2_SAP_4,Duplicate_1,how ='left', on = ['COUNTRY', 'VENDR_NBR', 'Month', 'Year','AMOUNT','DEAL_STATUS'])



In [ ]:
# R04 Round_Dollar
# US,MX,UK,CN,SA ---  round dollar in thousand
# AR,CL,CAM,IN,JP---  round dollar in ten thousand
df_db2_SAP_4['R04 Round_Dollar'] =0
country_bool_1 = [df_db2_SAP_4['COUNTRY'][i] in ['US','MX','UK','CN','SA','CA'] for i in df_db2_SAP_4.index]
country_bool_2 = [df_db2_SAP_4['COUNTRY'][i] in ['AR','CHILE','CAM','IN','JP'] for i in df_db2_SAP_4.index]
round_bool_1 = (df_db2_SAP_4['AMOUNT']/1000).apply(round)*1000 ==df_db2_SAP_4['AMOUNT']
round_bool_2 = (df_db2_SAP_4['AMOUNT']/10000).apply(round)*10000 ==df_db2_SAP_4['AMOUNT']

round_ix_1 = df_db2_SAP_4[(country_bool_1)&(round_bool_1)].index
round_ix_2 = df_db2_SAP_4[(country_bool_2)&(round_bool_2)].index

df_db2_SAP_4.loc[round_ix_1,'R04 Round_Dollar'] =1
df_db2_SAP_4.loc[round_ix_2,'R04 Round_Dollar'] = 1


In [ ]:
# R05 Buyer_Lst_5_days
# R06 Vendor_Lst_5_days
# R07 DMM_Lst_5_days
# R08 AR_Lst_5_days
# R09 All_Lst_5_days

def lst_day(x):
    if x =='?' or x =='Unknown':gap = 1000
    else:
        yr = x.year
        month = x.month
        if month ==12: lst_day = (date(yr+1, 1, 1) - date(yr, month, 1)).days
        else: lst_day = (date(yr, month+1, 1) - date(yr, month, 1)).days
        gap = lst_day - x.day
    return(gap)

Appr_Date_col = ['BUYER_APPROVAL_DATE','VENDOR_APPROVAL_DATE','DMM_APPROVAL_DATE','AR_APPROVAL_DATE']
for i in range(4):
    #print(i)
    risk_name = ''.join(['R0',str(i+5),' ',Appr_Date_col[i].split('_')[0],'_Lst_5_days'])
    df_db2_SAP_4[risk_name] = list(df_db2_SAP_4[Appr_Date_col[i]].apply(lst_day))
    
lst5day_col = [i for i in df_db2_SAP_4.columns  if 'Lst_5_days' in i]

temp = df_db2_SAP_4[lst5day_col]<5
all_appr5_bool = df_db2_SAP_4['R01_Tot_Appr_Levels'] == temp.apply(np.sum, axis=1)

df_db2_SAP_4['R09 All_Lst_5_days'] = 0
df_db2_SAP_4.loc[df_db2_SAP_4[all_appr5_bool].index,'R09 All_Lst_5_days'] =1



In [ ]:
# R10 BuyerInAR
# R11 VendorInAR
# R12 DMMInAR

BuyerInAR =  set(np.unique(df_db2_SAP_4['BUYER_ID'])).intersection(np.unique(df_db2_SAP_4['AR_ID']))

VendorInAR = set(np.unique(df_db2_SAP_4['VENDOR_ID'].dropna())).intersection(np.unique(df_db2_SAP_4['AR_ID']))

DMMInAR  = set(np.unique(df_db2_SAP_4['DMM_ID'].dropna())).intersection(np.unique(df_db2_SAP_4['AR_ID']))

df_db2_SAP_4['R10 BuyerInAR'] = df_db2_SAP_4['BUYER_ID'].apply(lambda x: x in BuyerInAR)
df_db2_SAP_4['R11 VendorInAR'] = df_db2_SAP_4['VENDOR_ID'].apply(lambda x: x in VendorInAR)
df_db2_SAP_4['R12 DMMInAR'] = df_db2_SAP_4['DMM_ID'].apply(lambda x: x in DMMInAR )




In [ ]:
# Write files for each country

os.chdir(r'\\phont80025us.homeoffice.wal-mart.com\Shares\Audit_Analytics\FY19\Projects\Global Vendor Allowance\Data\Coop_Deal_Raw_Data\Coop_Deal_Pro_Data')
print_ix = []
for i in df_db2_SAP_4['COUNTRY'].unique():
    df_db2_SAP_4[df_db2_SAP_4['COUNTRY']==i].to_csv(''.join(['Coop_Deal_w_Amt_Risk','_',i,'.csv']),index = False,encoding='utf_8_sig')
    #print_ix = print_ix + list(df_all_dsb[df_all_dsb['COUNTRY']==i].index)
    
    #print(''.join(['df_dsb','_',i,'.csv']))
